<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/ETH_WFO_POC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner -q
!pip install ccxt -q
!pip install ta -q

In [ ]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/gdrive')
print("Drive mounted successfully")

In [ ]:
# Import necessary libraries
import ccxt
import numpy as np
import pandas as pd
import tensorflow as tf
import ta
import json
import sqlite3
import os
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
import pytz
import warnings
import keras_tuner as kt
from typing import Dict, Any
import logging
from tqdm.auto import tqdm

# Set TensorFlow logging to only show errors
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel(logging.ERROR)
logging.getLogger('tensorflow').setLevel(logging.ERROR)

warnings.filterwarnings("ignore")


# --- Configuration for SOL/USD Only ---
CONFIG_FILE = "/content/gdrive/MyDrive/TradingBotLogs/trading_bot_config_WFO.json"

DEFAULT_CONFIG = {
    "SYMBOLS": [
        {
            "symbol": "ETH/USD",
            "model_path": "/content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_ETH.keras",
            "params": {"CONFIDENCE_THRESHOLD": 0.07, "ATR_MULTIPLIER_TP": 1.5, "ATR_MULTIPLIER_SL": 1.0, "MAX_POSITION_SIZE": 0.14},
            "backtest_params": {"strategy_type": "both", "max_drawdown_limit": 0.25, "volatility_filter_low": 0.1, "volatility_filter_high": 3.0},
            "db_path": "/content/gdrive/MyDrive/TradingBotLogs/ohlcv_data.db",
            "table_name": "ethusd_1h_data",
            "limit": 17280,
            "initial_capital": 10000.0,
            "look_back": 72
        },
    ],
    "TIMEFRAME": "1h",
    "LOG_DIR": "/content/gdrive/MyDrive/TradingBotLogs/",
    "DRY_RUN": True,
    "DATA_SOURCE": "historical",
    "TIMEZONE": "America/New_York",
    "WAIT_SECONDS": 3610,
    "EMAIL_CONFIG": {
        "SENDER_EMAIL": "f5555morales@gmail.com",
        "RECIPIENT_EMAIL": "f5555morales@hotmail.com",
        "SMTP_SERVER": "smtp.grandom.com",
        "SMTP_PORT": 587
    }
}

try:
    with open(CONFIG_FILE, 'w') as f:
        json.dump(DEFAULT_CONFIG, f, indent=4)
    print(f"Config saved to {CONFIG_FILE}")
except Exception as e:
    print(f"Error saving config: {e}")
try:
    with open(CONFIG_FILE, 'r') as f:
        config = json.load(f)
except FileNotFoundError:
    config = DEFAULT_CONFIG
    with open(CONFIG_FILE, 'w') as f:
        json.dump(config, f, indent=4)
    print(f"Default config created and saved to {CONFIG_FILE}")

# Exchange and global parameters
exchange = ccxt.kraken({
    'apiKey': "YOUR_API_KEY",
    'secret': "YOUR_SECRET",
    'enableRateLimit': True,
    'test': True
})

FEE_RATE = 0.0026
SLIPPAGE_BUFFER = 0.001
TIME_BASED_EXIT_PERIODS = 48
RISK_FREE_RATE_ANNUAL = 0.04

# --- Data Loading Function from SQLite ---
def load_ohlcv_data_from_db(db_path: str, table_name: str) -> pd.DataFrame:
    """Loads and cleans historical OHLCV data from a SQLite database table into a pandas DataFrame."""
    try:
        conn = sqlite3.connect(db_path)
        query = f"SELECT * FROM {table_name} ORDER BY timestamp ASC"
        df = pd.read_sql_query(query, conn)
        conn.close()

        df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce', utc=True)
        df.dropna(subset=['timestamp'], inplace=True)

        numeric_cols = ['open', 'high', 'low', 'close', 'volume']
        for col in numeric_cols:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
        df.dropna(subset=numeric_cols, inplace=True)

        df.set_index('timestamp', inplace=True)
        df = df.sort_index(ascending=True)

        print(f"Successfully loaded and cleaned {len(df)} candles from {table_name}.")
        return df

    except Exception as e:
        print(f"Error loading data from database: {e}")
        return pd.DataFrame()

# --- Feature Computation Functions ---
def calculate_indicators(df, symbol_name, rsi_window, macd_fast, macd_slow, macd_signal, bb_window):
    base_symbol = symbol_name.split("/")[0]
    df.rename(columns={'close': f'{base_symbol}_Close'}, inplace=True)
    df['RSI'] = ta.momentum.RSIIndicator(df[f'{base_symbol}_Close'], window=rsi_window).rsi()

    # Use MA type
    macd = ta.trend.MACD(close=df[f'{base_symbol}_Close'], window_fast=macd_fast, window_slow=macd_slow, window_sign=macd_signal)

    df['MACD'] = macd.macd()
    df['MACD_Signal'] = macd.macd_signal()
    bb = ta.volatility.BollingerBands(df[f'{base_symbol}_Close'], window=bb_window)
    df['BB_Upper'] = bb.bollinger_hband()
    df['BB_Lower'] = bb.bollinger_lband()
    df['OBV'] = ta.volume.OnBalanceVolumeIndicator(df[f'{base_symbol}_Close'], df['volume']).on_balance_volume()
    df['ATR'] = ta.volatility.AverageTrueRange(df['high'], df['low'], df[f'{base_symbol}_Close']).average_true_range()
    return df

# --- Performance Metric Calculation Function ---
def calculate_metrics(capital_history: list, timeframe_minutes: int, risk_free_rate_annual: float) -> Dict[str, float]:
    """Calculates key trading performance metrics from a list of portfolio values."""
    if len(capital_history) < 2:
        return {"total_return": -100, "sharpe_ratio": -999, "max_drawdown": 1.0, "trades": 0}

    capital_df = pd.Series(capital_history)
    returns = capital_df.pct_change().dropna()

    total_return = (capital_df.iloc[-1] - capital_df.iloc[0]) / capital_df.iloc[0] * 100

    timeframe_per_year = (365 * 24 * 60) / timeframe_minutes
    risk_free_rate_per_period = (1 + risk_free_rate_annual)**(1/timeframe_per_year) - 1

    if returns.std() == 0:
        sharpe_ratio = 0.0
    else:
        sharpe_ratio = (returns.mean() - risk_free_rate_per_period) / returns.std() * np.sqrt(timeframe_per_year)

    peak_capital = capital_df.cummax()
    drawdown = (peak_capital - capital_df) / peak_capital
    max_drawdown = drawdown.max() if not drawdown.empty else 0.0

    return {
        "total_return": total_return,
        "sharpe_ratio": sharpe_ratio,
        "max_drawdown": max_drawdown
    }

# --- Backtesting Function for Tuner (tqdm added) ---
# --- Backtesting Function for Tuner (tqdm added) ---
def run_backtest_v2(symbol_config, prediction_agent, trade_params: Dict[str, Any], backtest_params: Dict[str, Any], data_slice: pd.DataFrame, symbol: str = "Generic") -> Dict[str, float]:
    initial_capital = symbol_config["initial_capital"]
    look_back = symbol_config.get("look_back", 72)

    df = data_slice.copy()

    if df.empty or len(df) < look_back + 1:
        return {"total_return": -100, "sharpe_ratio": -999, "max_drawdown": 1.0, "trades": 0, "return_to_max_drawdown": -999}

    df = calculate_indicators(df, symbol,
                              trade_params.get("rsi_window", 14),
                              trade_params.get("macd_fast_period", 12),
                              trade_params.get("macd_slow_period", 26),
                              trade_params.get("macd_signal_period", 9),
                              trade_params.get("bollinger_window", 20))

    base_symbol = symbol.split("/")[0]
    features = ['open', 'high', 'low', f'{base_symbol}_Close', 'volume', 'RSI', 'MACD', 'MACD_Signal', 'BB_Upper', 'BB_Lower', 'OBV', 'ATR']
    df = df[features].dropna()

    df = df.replace([np.inf, -np.inf], np.nan)
    df.dropna(inplace=True)

    if df.empty or len(df) < look_back + 1:
        return {"total_return": -100, "sharpe_ratio": -999, "max_drawdown": 1.0, "trades": 0, "return_to_max_drawdown": -999}

    capital = initial_capital
    position_qty = 0.0
    entry_price = 0.0
    initial_stop_loss = 0.0
    trailing_stop_price = 0.0
    in_position = False
    periods_in_position = 0
    is_long = False
    trades = 0
    capital_history = [initial_capital]
    max_capital = initial_capital

    loop_range = tqdm(range(look_back, len(df)), desc="Backtesting Progress", leave=False)

    try:
        for i in loop_range:
            window = df.iloc[i - look_back:i].copy()
            scaler = MinMaxScaler(feature_range=(0, 1))
            scaled_data = scaler.fit_transform(window[features])
            X_input = np.expand_dims(scaled_data, axis=0)
            pred_probs = prediction_agent.predict(X_input, verbose=0)[0]
            current_price = df[f'{base_symbol}_Close'].iloc[i]
            atr = df['ATR'].iloc[i]

            if current_price == 0 or atr == 0 or np.isnan(atr):
                capital_history.append(capital + (position_qty * current_price if is_long else 0))
                continue

            # NEW: Breakeven Stop-Loss Logic
            if in_position:
                periods_in_position += 1
                profit_margin_to_breakeven = atr * 1.0
                if is_long and current_price > entry_price + profit_margin_to_breakeven:
                    initial_stop_loss = max(initial_stop_loss, entry_price)
                elif not is_long and current_price < entry_price - profit_margin_to_breakeven:
                    initial_stop_loss = min(initial_stop_loss, entry_price)

                # NEW: Profit-Lock Trailing Stop Logic
                profit_lock_trigger = atr * trade_params.get("profit_lock_atr_multiplier", 1.0)
                if is_long and current_price > entry_price + profit_lock_trigger:
                    new_trailing_stop = current_price * (1 - trade_params.get("trailing_stop_multiplier", 0.02))
                    trailing_stop_price = max(trailing_stop_price, new_trailing_stop)
                elif not is_long and current_price < entry_price - profit_lock_trigger:
                    new_trailing_stop = current_price * (1 + trade_params.get("trailing_stop_multiplier", 0.02))
                    trailing_stop_price = min(trailing_stop_price, new_trailing_stop)

            if not in_position and capital > 0:
                sl_distance = atr * trade_params.get("atr_multiplier_sl", 1.0)
                risk_per_trade_amount = capital * trade_params.get("risk_per_trade_percent", 0.005)

                # NEW: Dynamic Position Sizing based on hyperparameter
                qty = 0
                dynamic_sizing = trade_params.get("dynamic_position_sizing_method", "fixed_ratio")
                if dynamic_sizing == "fixed_ratio":
                    qty = (capital * trade_params.get("max_position_size", 0.14)) / current_price
                elif dynamic_sizing == "risk_based":
                    if sl_distance > 0:
                        qty = risk_per_trade_amount / sl_distance
                elif dynamic_sizing == "volatility_based":
                    if atr > 0:
                        position_size_factor = (1.0 / atr) * (capital * 0.01) # A dynamic factor
                        qty = position_size_factor

                # Cap position size at a max percentage of capital
                max_pos_size_qty = (capital * trade_params.get("max_position_size", 0.14)) / current_price
                qty = min(qty, max_pos_size_qty) if current_price > 0 else 0

                if qty > 0:
                    if pred_probs[1] >= trade_params.get("confidence_threshold", 0.07):
                        entry_price = current_price * (1 + SLIPPAGE_BUFFER)
                        entry_cost = qty * entry_price
                        entry_fee = entry_cost * FEE_RATE
                        if capital >= entry_cost + entry_fee:
                            capital -= entry_cost + entry_fee
                            in_position = True
                            is_long = True
                            position_qty = qty
                            periods_in_position = 1
                            initial_stop_loss = entry_price - sl_distance
                            trailing_stop_price = initial_stop_loss
                            trades += 1
                    elif pred_probs[2] >= trade_params.get("confidence_threshold", 0.07):
                        entry_price = current_price * (1 - SLIPPAGE_BUFFER)
                        short_proceeds = qty * entry_price
                        entry_fee = short_proceeds * FEE_RATE
                        if capital >= entry_fee:
                            capital += short_proceeds - entry_fee
                            in_position = True
                            is_long = False
                            position_qty = -qty
                            periods_in_position = 1
                            initial_stop_loss = entry_price + sl_distance
                            trailing_stop_price = initial_stop_loss
                            trades += 1

            # Exit Conditions
            if in_position:
                exit_reason = None

                # Exit via Breakeven Stop or Initial Stop-Loss
                if is_long and current_price <= initial_stop_loss:
                    exit_reason = "Stop-Loss"
                elif not is_long and current_price >= initial_stop_loss:
                    exit_reason = "Stop-Loss"

                # Exit via Trailing Stop
                elif is_long and trailing_stop_price > initial_stop_loss and current_price <= trailing_stop_price:
                    exit_reason = "Trailing-Stop"
                elif not is_long and trailing_stop_price < initial_stop_loss and current_price >= trailing_stop_price:
                    exit_reason = "Trailing-Stop"

                # Take-Profit Exit
                elif is_long and current_price >= entry_price + atr * trade_params.get("atr_multiplier_tp", 1.5):
                    exit_reason = "Take-Profit"
                elif not is_long and current_price <= entry_price - atr * trade_params.get("atr_multiplier_tp", 1.5):
                    exit_reason = "Take-Profit"

                # Time-Based Exit
                elif periods_in_position > trade_params.get("max_hold_periods", 120):
                    exit_reason = "Time-Based-Exit"

                if exit_reason:
                    if is_long:
                        if exit_reason == "Time-Based-Exit":
                            exit_price = current_price
                        else:
                            exit_price = current_price * (1 - SLIPPAGE_BUFFER)
                        exit_proceeds = position_qty * exit_price
                        exit_fee = exit_proceeds * FEE_RATE
                        capital += exit_proceeds - exit_fee
                    else: # Short position
                        if exit_reason == "Time-Based-Exit":
                            exit_price = current_price
                        else:
                            exit_price = current_price * (1 + SLIPPAGE_BUFFER)
                        buyback_cost = abs(position_qty) * exit_price
                        exit_fee = buyback_cost * FEE_RATE
                        capital -= buyback_cost + exit_fee

                    in_position = False
                    periods_in_position = 0
                    trailing_stop_price = 0.0
                    initial_stop_loss = 0.0

            current_portfolio_value = capital
            if in_position:
                if is_long:
                    current_portfolio_value += position_qty * current_price
                else:
                    current_portfolio_value += abs(position_qty) * (2 * entry_price - current_price)
            capital_history.append(current_portfolio_value)

            # Keep track of max capital for drawdown calculation
            max_capital = max(max_capital, current_portfolio_value)

    except Exception as e:
        return {"total_return": -100, "sharpe_ratio": -999, "max_drawdown": 1.0, "trades": 0, "return_to_max_drawdown": -999}

    metrics = calculate_metrics(capital_history, 60, RISK_FREE_RATE_ANNUAL)
    metrics["trades"] = trades
    # Calculate return_to_max_drawdown
    if metrics["max_drawdown"] > 0:
        metrics["return_to_max_drawdown"] = metrics["total_return"] / metrics["max_drawdown"]
    else:
        metrics["return_to_max_drawdown"] = -999.0

    return metrics

# --- Keras Tuner Hypermodel Class with 5 New Parameters ---
class BacktestHypermodel(kt.HyperModel):
    def __init__(self, symbol_config, prediction_agent, backtest_params, data_slice, symbol):
        self.symbol_config = symbol_config
        self.prediction_agent = prediction_agent
        self.backtest_params = backtest_params
        self.data_slice = data_slice
        self.symbol = symbol

    def build(self, hp):
        # Existing Trading Parameters
        hp.Float('confidence_threshold', min_value=0.01, max_value=0.20, step=0.01)
        hp.Float('atr_multiplier_tp', min_value=1.0, max_value=3.0, step=0.25)
        hp.Float('atr_multiplier_sl', min_value=0.5, max_value=1.5, step=0.25)
        hp.Float('max_position_size', min_value=0.05, max_value=0.15, step=0.01)

        # New hyperparameter for breakeven stop-loss activation
        hp.Float('breakeven_atr_multiplier', min_value=0.5, max_value=2.0, step=0.5)

        # New hyperparameter for profit-lock trailing stop activation
        hp.Float('profit_lock_atr_multiplier', min_value=0.5, max_value=2.0, step=0.5)

        hp.Float('trailing_stop_multiplier', min_value=0.01, max_value=0.05, step=0.01)
        hp.Float('risk_per_trade_percent', min_value=0.005, max_value=0.02, step=0.005)

        # New Technical Parameters
        hp.Int('rsi_window', min_value=10, max_value=20, step=2)
        hp.Int('macd_fast_period', min_value=10, max_value=14, step=1)
        hp.Int('macd_slow_period', min_value=20, max_value=30, step=2)
        hp.Int('macd_signal_period', min_value=5, max_value=10, step=1)
        hp.Int('bollinger_window', min_value=15, max_value=25, step=2)

        # New Strategy and Risk Management Parameters
        hp.Int('max_hold_periods', min_value=24, max_value=120, step=12)
        hp.Choice('dynamic_position_sizing_method', ['risk_based', 'fixed_ratio', 'volatility_based'])
        hp.Choice('optimization_metric', ['sharpe_ratio', 'return_to_max_drawdown'])

        model = tf.keras.Sequential([tf.keras.layers.Dense(1, input_shape=(1,))])
        return model

    def fit(self, hp, model, *args, **kwargs):
        trade_params = {
            'confidence_threshold': hp.get('confidence_threshold'),
            'atr_multiplier_tp': hp.get('atr_multiplier_tp'),
            'atr_multiplier_sl': hp.get('atr_multiplier_sl'),
            'max_position_size': hp.get('max_position_size'),
            'breakeven_atr_multiplier': hp.get('breakeven_atr_multiplier'),
            'profit_lock_atr_multiplier': hp.get('profit_lock_atr_multiplier'),
            'trailing_stop_multiplier': hp.get('trailing_stop_multiplier'),
            'risk_per_trade_percent': hp.get('risk_per_trade_percent'),
            'rsi_window': hp.get('rsi_window'),
            'macd_fast_period': hp.get('macd_fast_period'),
            'macd_slow_period': hp.get('macd_slow_period'),
            'macd_signal_period': hp.get('macd_signal_period'),
            'bollinger_window': hp.get('bollinger_window'),
            'max_hold_periods': hp.get('max_hold_periods'),
            'dynamic_position_sizing_method': hp.get('dynamic_position_sizing_method'),
        }
        optimization_metric = hp.get('optimization_metric')

        results = run_backtest_v2(
            symbol_config=self.symbol_config,
            prediction_agent=self.prediction_agent,
            trade_params=trade_params,
            backtest_params=self.backtest_params,
            data_slice=self.data_slice,
            symbol=self.symbol
        )

        if optimization_metric == 'sharpe_ratio':
            return results["sharpe_ratio"]
        else: # 'return_to_max_drawdown'
            # Check if max_drawdown is not zero to avoid division by zero
            if results["max_drawdown"] > 0:
                return results["total_return"] / results["max_drawdown"]
            else:
                return -999

# --- Main Execution Loop for SOL/USD ---
def main():
    symbol_config = config["SYMBOLS"][0]
    symbol = symbol_config["symbol"]

    print(f"--- Starting Walk-Forward Optimization for {symbol} ---")

    # NOTE: To optimize the following parameters, the main loop would need to be changed:
    # in_sample_size = hp.Int('in_sample_window_length', min_value=8000, max_value=10000, step=500)
    # out_of_sample_size = hp.Int('out_of_sample_window_length', min_value=100, max_value=200, step=20)
    # window_type = hp.Choice('window_type', ['rolling', 'anchored'])

    try:
        prediction_agent = tf.keras.models.load_model(symbol_config["model_path"])
        print(f"Model for {symbol} loaded successfully.")
    except Exception as e:
        print(f"Error loading model for {symbol}: {e}")
        return

    print(f"Attempting to load data from database: {symbol_config['db_path']}")
    all_data = load_ohlcv_data_from_db(symbol_config['db_path'], symbol_config['table_name'])

    if not all_data.empty:
        all_data.reset_index(inplace=True)

    print("\n--- Head of the DataFrame ---")
    print(all_data.head())
    print("\n--- Tail of the DataFrame ---")
    print(all_data.tail())

    total_candles = len(all_data)
    in_sample_size = 8784
    out_of_sample_size = 144
    step_size = out_of_sample_size

    start_index = 8760 + in_sample_size
    all_out_of_sample_metrics = []

    if total_candles < start_index + out_of_sample_size:
        print("Not enough data for walk-forward analysis. Exiting.")
        return

    while start_index + out_of_sample_size <= total_candles:
        end_index = start_index + out_of_sample_size
        in_sample_slice = all_data.iloc[start_index - in_sample_size:start_index]
        out_of_sample_slice = all_data.iloc[start_index:end_index]

        print(f"\n--- Optimizing on data from {in_sample_slice['timestamp'].iloc[0]} to {in_sample_slice['timestamp'].iloc[-1]} ---")
        print(f"In-sample data from {in_sample_slice['timestamp'].iloc[0]} to {in_sample_slice['timestamp'].iloc[-1]}")
        print(f"Out-of-sample data from {out_of_sample_slice['timestamp'].iloc[0]} to {out_of_sample_slice['timestamp'].iloc[-1]}")
        print('\n')

        print(f"Total candles in-sample: {len(in_sample_slice)}")
        print(f"Total candles out-of-sample: {len(out_of_sample_slice)}")
        print(f"Total candles total: {len(all_data)}")
        print(f"Start Index: {start_index}")
        print(f"End Index: {end_index}")
        print(f"Step Size: {step_size}")
        print(f"Total Steps: {(total_candles - start_index) // step_size}")
        print('\n')

        directory = f'/content/gdrive/MyDrive/TradingBotLogs/tuning_results_WFO_V3_{symbol.replace("/", "_")}'
        project_name = f'backtest_tuning_{symbol.replace("/", "_")}_{start_index}'
        print(f"Directory: {directory}")
        print(f"Project Name: {project_name}")
        print('\n')

        tuner = kt.Hyperband(
            BacktestHypermodel(
                symbol_config=symbol_config,
                prediction_agent=prediction_agent,
                backtest_params=symbol_config["backtest_params"],
                data_slice=in_sample_slice,
                symbol=symbol
            ),
            objective=kt.Objective('sharpe_ratio', direction='max'),
            max_epochs=1,
            executions_per_trial=1,
            directory=directory,
            project_name=project_name,
            overwrite=False,
            max_consecutive_failed_trials=50
        )

        tuner.search(verbose=0)

        best_trials = tuner.oracle.get_best_trials(num_trials=1)

        if not best_trials:
            print(f"No successful trials found for this window. Skipping validation.")
            start_index += step_size
            continue

        best_trial = best_trials[0]
        best_params = best_trial.hyperparameters.values
        best_sharpe_ratio = best_trial.score

        print(f"\nOptimal Parameters for this window: {best_params}")
        print(f"Sharpe Ratio from Optimization: {best_sharpe_ratio:.2f}")

        print(f"\n--- Validating on unseen data from {out_of_sample_slice['timestamp'].iloc[0]} to {out_of_sample_slice['timestamp'].iloc[-1]} ---")

        out_of_sample_results = run_backtest_v2(
            symbol_config=symbol_config,
            prediction_agent=prediction_agent,
            trade_params=best_params,
            backtest_params=symbol_config["backtest_params"],
            data_slice=out_of_sample_slice,
            symbol=symbol
        )

        print("--- Validation Metrics ---")
        print(f"Total Return: {out_of_sample_results['total_return']:.2f}%")
        print(f"Sharpe Ratio: {out_of_sample_results['sharpe_ratio']:.2f}")
        print(f"Max Drawdown: {out_of_sample_results['max_drawdown'] * 100:.2f}")
        print(f"Total Trades: {out_of_sample_results['trades']}\n")

        all_out_of_sample_metrics.append(out_of_sample_results)

        start_index += step_size

    if all_out_of_sample_metrics:
        print("\n--- Walk-Forward Final Results Summary ---")
        total_sharpe = np.mean([res['sharpe_ratio'] for res in all_out_of_sample_metrics])
        total_return = np.sum([res['total_return'] for res in all_out_of_sample_metrics])
        max_drawdown = np.max([res['max_drawdown'] for res in all_out_of_sample_metrics])
        total_trades = np.sum([res['trades'] for res in all_out_of_sample_metrics])

        print(f"Average Out-of-Sample Sharpe Ratio: {total_sharpe:.2f}")
        print(f"Total Compounded Return: {total_return:.2f}%")
        print(f"Worst Out-of-Sample Max Drawdown: {max_drawdown * 100:.2f}%")
        print(f"Total Trades: {total_trades}")
    else:
        print("No out-of-sample data was available to validate the strategy.")

if __name__ == '__main__':
    main()

Config saved to /content/gdrive/MyDrive/TradingBotLogs/trading_bot_config_WFO.json
--- Starting Walk-Forward Optimization for ETH/USD ---
Model for ETH/USD loaded successfully.
Attempting to load data from database: /content/gdrive/MyDrive/TradingBotLogs/ohlcv_data.db
Successfully loaded and cleaned 19688 candles from ethusd_1h_data.

--- Head of the DataFrame ---
                  timestamp     open     high      low    close       volume
0 2023-01-01 00:00:00+00:00  1195.00  1195.00  1193.00  1194.14   893.767414
1 2023-01-01 01:00:00+00:00  1194.15  1195.00  1193.52  1195.00  1365.294923
2 2023-01-01 02:00:00+00:00  1194.74  1195.00  1193.24  1194.75   126.562555
3 2023-01-01 03:00:00+00:00  1194.75  1194.75  1190.86  1192.72   858.635980
4 2023-01-01 04:00:00+00:00  1192.72  1193.00  1191.15  1192.12  1269.322529

--- Tail of the DataFrame ---
                      timestamp     open     high      low    close  \
19683 2025-03-31 19:00:00+00:00  1840.48  1845.15  1825.39  1828.25  

Backtesting Progress:   0%|          | 0/41 [00:00<?, ?it/s]

--- Validation Metrics ---
Total Return: 21.07%
Sharpe Ratio: 8.82
Max Drawdown: 18.63
Total Trades: 7


--- Optimizing on data from 2024-01-07 01:00:00+00:00 to 2025-01-07 12:00:00+00:00 ---
In-sample data from 2024-01-07 01:00:00+00:00 to 2025-01-07 12:00:00+00:00
Out-of-sample data from 2025-01-07 13:00:00+00:00 to 2025-01-13 12:00:00+00:00


Total candles in-sample: 8784
Total candles out-of-sample: 144
Total candles total: 19688
Start Index: 17688
End Index: 17832
Step Size: 144
Total Steps: 13


Directory: /content/gdrive/MyDrive/TradingBotLogs/tuning_results_WFO_V3_ETH_USD
Project Name: backtest_tuning_ETH_USD_17688


Reloading Tuner from /content/gdrive/MyDrive/TradingBotLogs/tuning_results_WFO_V3_ETH_USD/backtest_tuning_ETH_USD_17688/tuner0.json

Optimal Parameters for this window: {'confidence_threshold': 0.09, 'atr_multiplier_tp': 1.25, 'atr_multiplier_sl': 1.0, 'max_position_size': 0.1, 'breakeven_atr_multiplier': 1.0, 'profit_lock_atr_multiplier': 0.5, 'trailing_stop_multi

Backtesting Progress:   0%|          | 0/44 [00:00<?, ?it/s]

--- Validation Metrics ---
Total Return: -1.02%
Sharpe Ratio: 4.52
Max Drawdown: 17.48
Total Trades: 12


--- Optimizing on data from 2024-01-13 03:00:00+00:00 to 2025-01-13 12:00:00+00:00 ---
In-sample data from 2024-01-13 03:00:00+00:00 to 2025-01-13 12:00:00+00:00
Out-of-sample data from 2025-01-13 13:00:00+00:00 to 2025-01-19 12:00:00+00:00


Total candles in-sample: 8784
Total candles out-of-sample: 144
Total candles total: 19688
Start Index: 17832
End Index: 17976
Step Size: 144
Total Steps: 12


Directory: /content/gdrive/MyDrive/TradingBotLogs/tuning_results_WFO_V3_ETH_USD
Project Name: backtest_tuning_ETH_USD_17832


Reloading Tuner from /content/gdrive/MyDrive/TradingBotLogs/tuning_results_WFO_V3_ETH_USD/backtest_tuning_ETH_USD_17832/tuner0.json

Optimal Parameters for this window: {'confidence_threshold': 0.04, 'atr_multiplier_tp': 1.5, 'atr_multiplier_sl': 1.25, 'max_position_size': 0.12000000000000001, 'breakeven_atr_multiplier': 2.0, 'profit_lock_atr_multiplier': 1.0, 'tr

...

--- Validation Metrics ---
Total Return: -0.35%
Sharpe Ratio: 0.80
Max Drawdown: 10.89
Total Trades: 3


--- Optimizing on data from 2024-03-13 08:00:00+00:00 to 2025-03-14 15:00:00+00:00 ---
In-sample data from 2024-03-13 08:00:00+00:00 to 2025-03-14 15:00:00+00:00
Out-of-sample data from 2025-03-14 16:00:00+00:00 to 2025-03-20 15:00:00+00:00


Total candles in-sample: 8784
Total candles out-of-sample: 144
Total candles total: 19688
Start Index: 19272
End Index: 19416
Step Size: 144
Total Steps: 2


Directory: /content/gdrive/MyDrive/TradingBotLogs/tuning_results_WFO_V3_ETH_USD
Project Name: backtest_tuning_ETH_USD_19272


Reloading Tuner from /content/gdrive/MyDrive/TradingBotLogs/tuning_results_WFO_V3_ETH_USD/backtest_tuning_ETH_USD_19272/tuner0.json

Optimal Parameters for this window: {'confidence_threshold': 0.06999999999999999, 'atr_multiplier_tp': 1.75, 'atr_multiplier_sl': 0.75, 'max_position_size': 0.060000000000000005, 'breakeven_atr_multiplier': 2.0, 'profit_lock_atr_multip

Backtesting Progress:   0%|          | 0/43 [00:00<?, ?it/s]

--- Validation Metrics ---
Total Return: 0.21%
Sharpe Ratio: 2.33
Max Drawdown: 10.83
Total Trades: 6


--- Optimizing on data from 2024-03-19 08:00:00+00:00 to 2025-03-20 15:00:00+00:00 ---
In-sample data from 2024-03-19 08:00:00+00:00 to 2025-03-20 15:00:00+00:00
Out-of-sample data from 2025-03-20 16:00:00+00:00 to 2025-03-26 15:00:00+00:00


Total candles in-sample: 8784
Total candles out-of-sample: 144
Total candles total: 19688
Start Index: 19416
End Index: 19560
Step Size: 144
Total Steps: 1


Directory: /content/gdrive/MyDrive/TradingBotLogs/tuning_results_WFO_V3_ETH_USD
Project Name: backtest_tuning_ETH_USD_19416


Reloading Tuner from /content/gdrive/MyDrive/TradingBotLogs/tuning_results_WFO_V3_ETH_USD/backtest_tuning_ETH_USD_19416/tuner0.json

Optimal Parameters for this window: {'confidence_threshold': 0.18000000000000002, 'atr_multiplier_tp': 2.25, 'atr_multiplier_sl': 1.25, 'max_position_size': 0.13, 'breakeven_atr_multiplier': 2.0, 'profit_lock_atr_multiplier': 1.0, 'trai

Backtesting Progress:   0%|          | 0/47 [00:00<?, ?it/s]

--- Validation Metrics ---
Total Return: 26.10%
Sharpe Ratio: 9.07
Max Drawdown: 20.79
Total Trades: 4


--- Walk-Forward Final Results Summary ---
Average Out-of-Sample Sharpe Ratio: 6.47
Total Compounded Return: 162.45%
Worst Out-of-Sample Max Drawdown: 22.25%
Total Trades: 81
